In [10]:
import geojson
import pandas as pd
from shapely.geometry import Polygon, Point
import json
import geopy
import geopy.distance
import rasterio as rio
from pathlib import Path
import os
from rasterio import mask


DISTANCE = 2
solar_pan_type = "below_threshold"


In [11]:
def find_file_by_point(lon, lat, dictionary):
    for key, val in dictionary.items():
        if Polygon(val).contains(Point(lat, lon)):
            return key
    return None

def polygon_definition(lon,lat,distance):
    polygon_list = []
    start = geopy.Point(lat,lon)
    r = geopy.distance.distance(kilometers = distance/2)
    d = geopy.distance.distance(kilometers = distance)
    new_point = r.destination(point=start, bearing=0)
    polygon_1 = r.destination(point=new_point,bearing=90)
    coordinates = polygon_1[1],polygon_1[0]
    polygon_list.append(tuple(coordinates))
    for i in range(3):
        polygon_point = d.destination(point=polygon_1,bearing=(90*(i+2)))
        polygon_1 = polygon_point
        coordinates = polygon_1[1],polygon_1[0]
        polygon_list.append(tuple(coordinates))
    polygon_list.append(polygon_list[0])
    return polygon_list

In [12]:

with open('recog.json', 'r') as json_file:
    data = json.load(json_file)

df = pd.read_csv("data\\solar_park_dataset_{}.csv".format(solar_pan_type), sep=";")
for i in range(len(df)):
    lat = df["Latitude"][i]
    lon = df["Longitude"][i]
    GEM_ID = df["GEM phase ID"][i]
    country = df["Country"][i]
    file_name = find_file_by_point(lon,lat,data)
    geom_list = polygon_definition(lon,lat,DISTANCE)
    file_path = os.path.join(Path.cwd().parent, 'raw_DEM\\{}'.format(file_name))


    geometry = geojson.Polygon([geom_list])

    with rio.open("{}".format(file_path)) as src:
        out_image, out_transform = rio.mask.mask(src, [geometry],crop=True, all_touched=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        src.close()

    with rio.open(os.path.join(Path.cwd(),"Masked_images_dic\\{}\\{}_{}_masked.tiff".format(solar_pan_type,GEM_ID,country)), "w", **out_meta) as dest:
        dest.write(out_image)
        dest.close()
